# Unit testing

Two main purposes:

1. Describe what your application does and what your functions in your application do. By looking at your tests, other people should get a general idea of what your code does, some of the classes and functions, their limits, etc.
2. **Checks** are something that run on your application code and check for the same thing. A **test** aims to check if your application will break at any point, checks for problems, etc. There is debate as to how much the two overlap.
3. It forces you to write better code.

Unit testing isn't necessary for small projects or portfolio piece. If the project continues to grow, tests are important.

## Testing functions

In [1]:
# test_functions.py
from unittest import TestCase

class TestFunctions(TestCase):
    def test_divide_result(self):
        divident = 15
        divisor = 3
        expected_result = 5.0
        self.assertEqual(divide(divident, divisor), expected_result)

In `self.assertEqual()`, you can also set `delta=0.0001` to allow for different expected results.

To run this code, you must have the function and file name start with `test_`. Then you can right click the file and click Run, which will run the unit tests. Note that the python files have the same name except for the prepended `test_`.

In [2]:
# functions.py
def divide(dividend, divisor):
    if divisor == 0:
        raise ValueError('The divisor cannot be zero.')
    
    return dividend / divisor


def multiply(*args):
    if len(args) == 0:
        raise ValueError('At least one value to multiply must be passed.')
    total = 1
    for arg in args:
        total *= arg
    
    return total

You can add more tests to the class `TestFunctions` to perform multiple tests.

## Testing for errors

To test for an error, you could add the method to the class from above:

In [4]:
# test_functions.py
from unittest import TestCase

class TestFunctions(TestCase):
    def test_divide_result(self):
        divident = 15
        divisor = 3
        expected_result = 5.0
        self.assertEqual(divide(divident, divisor), expected_result)
    
    def test_divide_error_on_zero(self):
        with self.assertRaises(ValueError):
            divide(25, 0)
    
    # tests empty arguments
    def test_multiply_empty(self):
        with self.assertRaises(ValueError):
            multiply()
    
    # test one item in list
    def test_multiply_single_value(self):
        expected = 15
        self.assertEqual(multiply(expected), expected)
    
    # test results with unpacking arguments
    def test_multiply_result(self):
        inputs = (3, 5)
        expected = 15
        self.assertEqual(multiply(*inputs), expected)

# Testing classes

In [5]:
# printer.py
class PrinterError(RuntimeError):
    pass


class Printer:
    def __init__(self, pages_per_s: int, capacity: int):
        self.pages_per_s = pages_per_s
        self._capacity = capacity
    
    def print(self, pages):
        if pages > self._capacity:
            raise PrinterError('Printer does not have enough capacity for all these pages.')
        
        self._capacity -= pages

        return f'Printed {pages} pages in {pages/self.pages_per_s:.2f} seconds.'

In [ ]:
# test_printer.py
from unittest import TestCase


class TestPrinter(TestCase):
    # creates a Printer instance to test for all other tests
    # runs before each test, so each time you get a new Printer
    # if you want to keep the same Printer, use `@ classmethod \n def setUpClass(cls): ...`
    def setUp(self):
        self.printer = Printer(pages_per_s=2.0, capacity=300)
    
    def test_print_within_capacity(self):
        self.printer.print(25)
    
    # same format as functions
    def test_print_outside_capacity(self):
        with self.assertRaises(PrinterError):
            self.printer.print(301)
    
    # make sure edge case is checked
    def test_print_exact_capacity(self):
        self.printer.print(self.printer._capacity)
    
    def test_printer_speed(self):
        pages = 10
        expected = 'Printed 10 pages in 5.00 seconds.'

        result = self.printer.print(pages)
        self.assertEqual(result, expected)
    
    def test_speed_always_two_decimals(self):
        fast_printer = Printer(pages_per_s=3.0, capacity=300)
        pages = 11
        expected = 'Printed 11 pages in 3.67 seconds.'

        result = fast_printer.print(pages)
        self.assertEqual(result, expected)

    def test_multiple_print_runs(self):
        self.printer.print(25)
        self.printer.print(50)
        self.printer.print(225)
    
    def test_multiple_runs_end_up_error(self):
        self.printer.print(25)
        self.printer.print(50)
        self.printer.print(225)

        with self.assertRaises(PrinterError):
            self.printer.print(1)

## Testing external libraries

In [8]:
# page.py
import requests

class PageRequester:
    def __init__(self, url):
        self.url = url
    
    def get(self):
        return requests.get(self.url)

In [7]:
# test_page.py
from unittest import TestCase
from unittest.mock import patch


class TestPageRequester(TestCase):
    def setUp(self):
        self.page = PageRequester('https://google.com')
    
    # anywhere in our application, 'requests.get' is replaced by a mock
    def test_make_request(self):
        with patch('requests.get') as mocked_get:
            self.page.get()
            mocked_get.assert_called()
    
    def test_content_returned(self):
        class FakeResponse:
            def __init__(self):
                self.content = 'Hello'

        with patch('requests.get', return_value=FakeResponse()) as mocked_get:
            result = self.page.get()
            self.assertEqual(result, 'Hello')

By replacing the library functionality with a `MagicMock`, this allows us to always access any property it invents, despite if it exists or not. The property will return another `MagicMock`. Our patch returns a class we created called `FakeResponse`, which does have a `content` property. We are essentially faking a return value to test the rest of our code that surrounds an external library.